In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import os

%matplotlib inline
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

In [ ]:
# trajectory model to test.

#trajectory_type = 'circle'  # has constant velocity in model.
#trajectory_type = 'ellipse' # non-constant velocity.
#trajectory_type = 'linear' # linear constant velocity
trajectory_type = 'linear_accel' # linear constant acceleration

In [ ]:
from trajectory_creator import get_trajectory
from trajectory import Trajectory

n_times = 21

if trajectory_type == 'circle':
    trajectory = get_trajectory('circle2_double.csv')
    lengths_uniform = r * 2 * np.pi / (n_times - 1)
    print('uniform length:', lengths_uniform)
    max_time = trajectory.params['tau']
    
if trajectory_type == 'ellipse':
    trajectory = get_trajectory('circle2_double.csv')
    trajectory.set_coeffs(seed=2)
    max_time = trajectory.params['tau']
    
if trajectory_type == 'linear':
    trajectory = get_trajectory('straight1.csv')
    max_time = 1.0
    
if trajectory_type == 'linear_accel':
    trajectory = get_trajectory('straight1.csv')
    trajectory = Trajectory(n_complexity=3, dim=2, model='polynomial')
    trajectory.set_coeffs(coeffs=np.array([[0, 1, 1], [0, 0.5, 0.5]]))
    max_time = 1.0
    
plt.figure()
times_uniform = np.linspace(0, max_time, n_times)
basis = trajectory.get_basis(times=times_uniform)
trajectory.plot(times=times_uniform)
plt.axis('equal')

In [ ]:
lengths_noncum = trajectory.get_distances_from_times(times_uniform)

if np.allclose(lengths_noncum, lengths_noncum[0]):
    print('constant velocity:', lengths_noncum[0])
else:
    print('non-constant velocity')

lengths_cum = np.cumsum(lengths_noncum)
print('cumulative lengths:', lengths_cum)
times_prime, __, __ = trajectory.get_times_from_distances(arbitrary_distances=lengths_cum)

np.testing.assert_allclose(times_prime, times_uniform, rtol=1e-3, atol=1e-3)
print('test 1 passed.')

In [ ]:
# for linear acceleration only
if trajectory_type != 'linear_accel':
    print('Nothing else to do here.')
else:
    # convert the nonlinear times to times at constant velocity (such as what we get from robot). 
    new_times = times_uniform
    points = trajectory.get_sampling_points(times=new_times)
    direction = points[:, -1] - points[:, 0]
    points_robot = points[:, 0] + np.linspace(0, 1, n_times).reshape((-1, 1)) * direction
    points_robot = points_robot.T
    
    fig, ax = plt.subplots(2, 3); fig.set_size_inches(13, 4)
    ax[0, 0].scatter(*points[:2])
    ax[0, 0].set_title('model points')
    ax[0, 1].scatter(*points_robot[:2])
    ax[0, 1].set_title('robot measurement points')
    ax[1, 0].scatter(new_times, y=[1]*len(new_times))
    ax[1, 1].scatter(new_times, y=[1]*len(new_times))
    
    constant_length = np.linalg.norm(points_robot[:, 0] - points_robot[:, 1])
    constant_velocity = constant_length / (new_times[1] - new_times[0])
    
    lengths = [constant_length] * (n_times - 1)
    
    times_prime, __, __ = trajectory.get_times_from_distances(arbitrary_distances=np.cumsum(lengths))
    
    sampling_points = trajectory.get_sampling_points(times=times_prime)
    ax[0, 2].scatter(*sampling_points[:2])
    ax[0, 2].set_title('sampling points')
    ax[1, 2].scatter(times_prime, y=[1]*len(times_prime))
    
    for i in range(3):
        ax[0, i].set_xlabel('x');  
        ax[0, i].set_ylabel('y'); 
        ax[1, i].set_ylabel('')
        ax[1, i].set_yticklabels('')
        ax[1, i].set_xlabel('times')
    
    np.testing.assert_allclose(sampling_points, points_robot, rtol=1e-3, atol=1e-3)
    print('test 2 passed.')
